In [ ]:
import pandas as pd
import numpy as np
import string
import csv
import urllib
import requests 

In [ ]:
import os

# modify the line below and add the path to your google credentials json file
# further information on how to generate such a json file: https://cloud.google.com/docs/authentication/getting-started#command-line
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = PLACEHOLDER_APPLICATION_CREDENTIALS

In [ ]:
from google.cloud import storage

# in this notebook we need two Google Cloud Storage buckets: https://cloud.google.com/storage/docs/creating-buckets
# in the first bucket we have the videos
# in the second bucket we store the labels extracted from the video frames

# Replace all PLACEHOLDER_ variables with the variables you have created.
def list_blobs_with_prefix(bucket_name, prefix, delimiter):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    blobs = bucket.list_blobs(prefix=prefix, delimiter=delimiter)

    videos = []
    for blob in blobs:
        if not blob.name.endswith("/"):
            videos.append("gs://" + bucket_name + "/" + blob.name)
            
    return videos
            
videos = list_blobs_with_prefix(PLACEHOLDER_BUCKET_NAME_VIDEOS, PLACEHOLDER_PREFIX, PLACEHOLDER_DELIMITER)
            

In [ ]:
from google.cloud import videointelligence
import csv

storage_client = storage.Client()

# Replace all PLACEHOLDER_ variables with the variables you have created.
gcloud_bucket_name_frame = PLACEHOLDER_BUCKET_NAME_VIDEO_LABELS
bucket_frame = storage_client.get_bucket(gcloud_bucket_name_frame)

""" Detects labels given a GCS path. """
video_client = videointelligence.VideoIntelligenceServiceClient()
features = [videointelligence.enums.Feature.LABEL_DETECTION]

mode = videointelligence.enums.LabelDetectionMode.SHOT_AND_FRAME_MODE
config = videointelligence.types.LabelDetectionConfig(
    label_detection_mode=mode)
context = videointelligence.types.VideoContext(
    label_detection_config=config)

for video in videos:
    
    videoTitle = video.split("/")[-1]

    names = ["storageLink","label","time_offset","confidence","type", "label_entity_id", "type_id"]
    with open("../data/labels_video_frames/" + videoTitle + ".csv", "w" ) as outSentences:
        writer = csv.writer( outSentences )
        writer.writerow(names)

    operation = video_client.annotate_video(
        video, features=features, video_context=context)
    
    # Process frame level label annotations
    frame_labels = result.annotation_results[0].frame_label_annotations
    for i, frame_label in enumerate(frame_labels):
        categories = []
        category_ids = []
        for category_entity in frame_label.category_entities:
            categories.append(category_entity.description)
            category_ids.append(category_entity.entity_id)

        for i, frame in enumerate(frame_label.frames):
            time_offset = (frame.time_offset.seconds + frame.time_offset.nanos / 1e9)
            confidence = frame.confidence

            row = []
            row.append(videoTitle[0:-4])
            row.append(frame_label.entity.description)
            row.append(time_offset)
            row.append(confidence)
            row.append(", ".join(categories))
            row.append(frame_label.entity.entity_id)
            row.append(", ".join(category_ids))

            with open("../data/labels_video_frames/" + videoTitle + ".csv", "a" ) as outSentences:
                writer = csv.writer( outSentences )
                writer.writerow(row)


    blob = bucket_frame.blob("" + videoTitle + ".csv")
    blob.upload_from_filename("../data/labels_video_frames/" + videoTitle + ".csv")